## **Ingesta del directorio `lap_times`**

In [ ]:
# Creamos un parametro para el nombre del archivo
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [ ]:
# Creamos un parametro para la fecha del archivo
dbutils.widgets.text("p_file_date", "")
v_file_date = dbutils.widgets.get("p_file_date")

In [ ]:
# Llamamos al notebook que contiene las variables de configuración
%run "../utils/configuration"

In [ ]:
# Llamamos al notebook que contiene funciones comunes
%run "../utils/common_functions"

### Paso 1 - Crear tabla **f1_raw.lap_times**

In [ ]:
%sql
DROP TABLE IF EXISTS f1_raw.lap_times;
CREATE EXTERNAL TABLE IF NOT EXISTS f1_raw.lap_times(
raceId INT,
driverId INT,
lap INT,
position INT,
time STRING,
milliseconds INT
)
USING CSV
OPTIONS (path "/mnt/formula1dl/raw/lap_times")

### Paso 2 - Leer el directorio **lap_times** el cual contiene multiples archivos CSV

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [ ]:
lap_times_schema = StructType(fields=[StructField("raceId", IntegerType(), False),
                                      StructField("driverId", IntegerType(), True),
                                      StructField("lap", IntegerType(), True),
                                      StructField("position", IntegerType(), True),
                                      StructField("time", StringType(), True),
                                      StructField("milliseconds", IntegerType(), True)
                                     ])

In [ ]:
# El parámetro "raw_folder_path" se encuentra en el notebook "configuration"
# El parámetro "v_file_date" se encuentra en el notebook e indicamos su valor en tiempo de ejecución
lap_times_df = spark.read \
.schema(lap_times_schema) \
.csv(f"{raw_folder_path}/lap_times")
#.csv(f"{raw_folder_path}/{v_file_date}/lap_times")

### Paso 3 - Renombrar columnas y añadir nuevas columnas

In [ ]:
from pyspark.sql.functions import lit

In [ ]:
# La función "add_ingestion_date()" se encuentra en el notebook "common_functions"
# La fecha de ingestión será del tipo timestamp
lap_times_with_ingestion_date_df = add_ingestion_date(lap_times_df)

In [ ]:
final_df = lap_times_with_ingestion_date_df.withColumnRenamed("driverId", "driver_id") \
                                           .withColumnRenamed("raceId", "race_id") \
                                           .withColumn("data_source", lit(v_data_source)) \
                                           .withColumn("file_date", lit(v_file_date))

### Paso 4 - Escribir datos en el datalake como parquet y crear la tabla **lap_times** en la base de datos **f1_processed**

In [ ]:
# Recordar que Databricks utiliza Hive para la gestión de tablas
# Esto quiere decir que una tabla en Hive se asocia con un directorio específico que contiene archivos de datos
# Aunque los datos subyacentes son solo archivos en un sistema de archivos, Hive proporciona una interfaz que presenta 
# estos datos como tablas.
# La base de datos tambien es un directorio. Podemos crearla manualmente, como con lenguaje SQL.
# Hive permite a los usuarios interactuar con esos datos utilizando un lenguaje similar a SQL
# En resumen, solo se esta creando un directorio y un archivo
# Escribimos los archivos del directorio "lap_times" con formato PARQUET en la base de datos "f1_processed" y en la tabla "lap_times"
final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_processed.lap_times")

In [ ]:
dbutils.notebook.exit("Success")